### r1 - part 1 : 挑出特定股票漲跌超過特定幅度的日期，並將結果存到 csv 中。
* input dataset: stock_data_2019-2023.csv
* output dataset: f{特定股票}.csv

In [4]:
import pandas as pd
import datetime

import warnings
warnings.filterwarnings('ignore')

In [5]:
def filter_target_stock(se_stock, target_stock, mode):
    """ filter target stock from datasets """
    stockdata = se_stock
    
    stockdata = stockdata[['證券代碼', '年月日', '收盤價(元)']]
    stockdata.rename(columns={'證券代碼': '證券代碼', '年月日': 'Date', '收盤價(元)': 'Close'}, inplace=True)
    stockdata = stockdata[stockdata['證券代碼'].str.contains(target_stock)]
    print("篩選出指定證券")
    display(stockdata)
    
    return stockdata
    
def filter_stockdata_by_up_and_down(stockdata, days=3, rate=0.05):
    #將資料中的日期欄位轉換為日期格式
    stockdata['Date'] = pd.to_datetime(stockdata['Date']).dt.date

    #根據日期欄位進行排序
    stockdata = stockdata.sort_values(by = 'Date').reset_index(drop = True)
    
    # 計算經過三股票日漲跌幅比例
    stockdata['經過三股票日漲跌比例(%)'] = (stockdata['Close'].shift(-days) - stockdata['Close']) / stockdata['Close']

    # 給予標記，漲幅超過 rate 定義 "漲"，跌幅超過 rate 定義 "跌"
    stockdata['label'] = stockdata['經過三股票日漲跌比例(%)'].apply(lambda x: '跌' if x < -rate else '漲' if x > rate else 'None')
    withoutfiltering = stockdata
    
    #去除非"漲"、非"跌"的資料
    stockdata = stockdata[stockdata['label'].isin(['漲', '跌'])]
    print("漲跌日期數")
    print(stockdata['label'].value_counts())
    
    return stockdata, withoutfiltering

### 0418 testing

In [6]:
# 上市 stock exchange
se_2023 = pd.read_excel(r'./bda2023_mid_dataset/stock_data_2019-2023.xlsx', sheet_name="上市2023")
se_2022 = pd.read_excel(r'./bda2023_mid_dataset/stock_data_2019-2023.xlsx', sheet_name="上市2022")
se_2021 = pd.read_excel(r'./bda2023_mid_dataset/stock_data_2019-2023.xlsx', sheet_name="上市2021")
se_2020 = pd.read_excel(r'./bda2023_mid_dataset/stock_data_2019-2023.xlsx', sheet_name="上市2020")
se_stock = pd.concat([se_2023, se_2022, se_2021, se_2020])
# 上櫃 otc
otc_2023 = pd.read_excel(r'./bda2023_mid_dataset/stock_data_2019-2023.xlsx', sheet_name="上櫃2023")
otc_2022 = pd.read_excel(r'./bda2023_mid_dataset/stock_data_2019-2023.xlsx', sheet_name="上櫃2022")
otc_2021 = pd.read_excel(r'./bda2023_mid_dataset/stock_data_2019-2023.xlsx', sheet_name="上櫃2021")
otc_2020 = pd.read_excel(r'./bda2023_mid_dataset/stock_data_2019-2023.xlsx', sheet_name="上櫃2020")
otc_stock = pd.concat([otc_2023, otc_2022, otc_2021, otc_2020])

print("上市")
display(se_stock)
print("上櫃")
display(otc_stock)

上市


,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,0050 元大台灣50,2023/03/24,122,122.3,121.55,122.20,13919,1697778,10776,2298000,-,-
1,0051 元大中型100,2023/03/24,57.35,57.45,57.15,57.40,41,2369,149,16000,-,-
2,0052 富邦科技,2023/03/24,109.7,110.1,109.5,110.10,383,42150,489,57000,-,-
3,0053 元大電子,2023/03/24,61.2,61.3,61.1,61.30,5,322,1013,4988,-,-
4,0055 元大MSCI金融,2023/03/24,21.86,21.86,21.74,21.83,220,4804,319,75654,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
238120,TWN50 台灣50指數,2020/01/02,9382.64,9458.51,9376.5,9450.90,400657,37335137,173161,355830808,-,-
238121,Y5555 不含電子,2020/01/02,15185,15217.1,15170.1,15203.12,668539,20684777,258924,479766580,-,-
238122,Y8886 未含金電,2020/01/02,12558.5,12595.6,12546.8,12578.50,461643,16830608,196095,258088091,-,-
238123,Y8888 不含金融,2020/01/02,10199.1,10281.4,10199.1,10271.14,1840092,125646483,827519,489492844,-,-


上櫃


,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,006201 元大富櫃50,2023/03/24,18.4,18.49,18.36,18.40,53,990,51,21946,-,-
1,00679B 元大美債20年,2023/03/24,32.14,32.14,32.02,32.10,14029,450159,6365,1996192,-,-
2,00687B 國泰20年美債,2023/03/24,33.59,33.68,33.57,33.68,2078,69872,415,297287,-,-
3,00694B 富邦美債1-3,2023/03/24,38.61,38.61,38.56,38.57,201,7755,19,117076,-,-
4,00695B 富邦美債7-10,2023/03/24,34.94,35.05,34.94,35.00,774,27134,35,22686,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
189303,OTC25 OTC 營造,2020/01/02,135.94,136.3,135.91,136.25,7351,118913,2423,3482058,-,-
189304,OTC26 OTC 運輸類,2020/01/02,71.07,71.26,70.98,71.11,1078,28491,511,559030,-,-
189305,OTC27 OTC 觀光類,2020/01/02,72.85,73.4,72.8,73.40,1403,82135,881,1285141,-,-
189306,OTC32 文化創意業指數,2020/01/02,102.62,104.57,102.62,102.82,25999,2628636,18336,1418825,-,-


In [7]:
mode = '上市'
target_stock = '2317 鴻海'
days = 3
rate = 0.05

stockdata = filter_target_stock(se_stock, target_stock, mode)
stockdata, withoutfiltering = filter_stockdata_by_up_and_down(stockdata, days, rate)

篩選出指定證券


,證券代碼,Date,Close
425,2317 鴻海,2023/03/24,105.5000
1686,2317 鴻海,2023/03/23,103.5000
2947,2317 鴻海,2023/03/22,103.5000
4208,2317 鴻海,2023/03/21,103.5000
5465,2317 鴻海,2023/03/20,102.5000
...,...,...,...
233512,2317 鴻海,2020/01/08,82.2707
234487,2317 鴻海,2020/01/07,84.7435
235462,2317 鴻海,2020/01/06,86.0751
236437,2317 鴻海,2020/01/03,87.1213


漲跌日期數
漲    28
跌    25
Name: label, dtype: int64


In [8]:
stockdata.to_csv(f'./bda2023_mid_dataset/up_and_down_dates/{target_stock}.csv')
withoutfiltering.to_csv(f'./bda2023_mid_dataset/without_filtering/{target_stock}.csv')